In [1]:
import os
import nest_asyncio
from llama_parse import LlamaParse
from llama_index.core.schema import ImageDocument, TextNode
from llama_index.multi_modal_llms.openai import OpenAIMultiModal
from typing import List

nest_asyncio.apply()

FILE_NAME = "./data/example2/past_elections.pdf"
IMAGES_DOWNLOAD_PATH = "./data/example2/images/"

LLAMA_CLOUD_API_KEY = os.environ["LLAMA_CLOUD_API_KEY"]
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [2]:
parser = LlamaParse(
    api_key=LLAMA_CLOUD_API_KEY,
    result_type="markdown",
)

json_objs = parser.get_json_result(FILE_NAME)
json_list = json_objs[0]["pages"]

Started parsing the file under job_id 2929e85a-de8d-4104-866b-81a07c320486


In [3]:
def get_text_nodes(json_list: List[dict]) -> List[TextNode]:
    return [TextNode(text=page["text"], metadata={"page": page["page"]}) for page in json_list]

text_nodes = get_text_nodes(json_list)

def get_image_nodes(json_objs: List[dict], download_path: str) -> List[ImageDocument]:
    image_dicts = parser.get_images(json_objs, download_path=download_path)
    return [ImageDocument(image_path=image_dict["path"]) for image_dict in image_dicts]

image_documents = get_image_nodes(json_objs, IMAGES_DOWNLOAD_PATH)

> Image for page 1: [{'name': 'img_p0_1.png', 'height': 303, 'width': 520, 'x': 177.479, 'y': 236.3814, 'original_width': 2560, 'original_height': 1489}, {'name': 'img_p0_2.png', 'height': 303, 'width': 520, 'x': 177.479, 'y': 430.99816, 'original_width': 1600, 'original_height': 931}]


In [6]:
openai_mm_llm = OpenAIMultiModal(
    model="gpt-4o", api_key=OPENAI_API_KEY, max_new_tokens=300
)

response = openai_mm_llm.complete(
    prompt=f"How does the image relate to the text: '{text_nodes[0].text}'? Which states switched from Trump to Biden between 2016 and 2020?",
    image_documents=image_documents,
)

print(response)


The images show the electoral maps for the 2016 and 2020 U.S. presidential elections. In these maps, red indicates states won by Trump, and blue indicates states won by Clinton (2016) or Biden (2020). 

Between 2016 and 2020, the states that switched from Trump to Biden are:

- Arizona
- Georgia
- Michigan
- Pennsylvania
- Wisconsin

These states were red in the 2016 map and blue in the 2020 map.
